In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

# Modeling
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
PATH_TRAIN = os.path.join('C:\\Users', 'Hugo Martins',
                       'PycharmProjects', 'CODES', 'train.csv')

PATH_TEST = os.path.join('C:\\Users', 'Hugo Martins',
                       'PycharmProjects', 'CODES', 'test.csv')

In [3]:
data_train = pd.read_csv(PATH_TRAIN)
data_test = pd.read_csv(PATH_TEST)
pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
data_train.head()
# data_train.columns

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
cols_miss_ls = [k for (k,v) in data_train.isnull().sum().to_dict().items() if v >=500]
data_train = data_train.drop(columns=cols_miss_ls)
data_test = data_test.drop(columns=cols_miss_ls)


In [5]:
cols_missing_tr = data_train.columns[data_train.isnull().any()]
cols_missing_te = data_test.columns[data_test.isnull().any()]
def imputation(dataframe, cols):
    mode_val = dataframe[cols].mode()
    if mode_val.empty:
        # Handle empty mode value DataFrame
        return dataframe
    else:
        # fill missing value with mode
        mode_val = mode_val.iloc[0]
        dataframe[cols] = dataframe[cols].fillna(mode_val)
    return dataframe
data_train.fillna(data_train.mode()).all()
data_tr_clean = imputation(data_train, cols_missing_tr)
data_te_clean = imputation(data_test, cols_missing_te)
# data_te_clean.isnull().sum()

In [6]:
data_categoric = data_tr_clean.select_dtypes(include=['object']).columns.tolist()
data_numeric = data_tr_clean.select_dtypes(include=['float', 'int']).drop(columns='SalePrice')
oe = OrdinalEncoder()
cat = oe.fit_transform(data_tr_clean[data_categoric])

In [7]:
target = data_tr_clean['SalePrice']
anova = SelectKBest(score_func=f_regression, k=30)
anova.fit_transform(cat,target)

anova_score = pd.DataFrame( {'P_value_anova': anova.pvalues_}, index=data_categoric)

anova_cols_cat = anova_score[anova_score.P_value_anova <= 0.05].index.to_list()
# anova_cols.extend(data_tr_clean.select_dtypes(include=['float', 'int']).columns.tolist())



In [8]:
anova = SelectKBest(score_func=f_regression, k=30)
anova.fit_transform(data_numeric,target)

anova_num = pd.DataFrame( {'P_value_anova': anova.pvalues_},
                           index=data_numeric.columns.tolist())

# anova_score.sort_values(by=['P_value_anova'], ascending=False)
anv_cols_num = anova_num[anova_num.P_value_anova <= 0.05].index.to_list()

use_cols = []
use_cols.extend(anova_cols_cat)
use_cols.extend(anv_cols_num)   

data_train_cl = data_tr_clean[use_cols]
data_test_cl = data_te_clean[use_cols]


In [9]:

minmax = MinMaxScaler()

train_scaled = minmax.fit_transform(data_tr_clean[anv_cols_num])
test_scaled = minmax.transform(data_test_cl[anv_cols_num])

In [10]:
oe = OrdinalEncoder()
train_encoded = oe.fit_transform(data_tr_clean[anova_cols_cat])
test_encoded = oe.fit_transform(data_test_cl[anova_cols_cat])

In [11]:
data_f = pd.DataFrame(np.concatenate((train_scaled, train_encoded), axis=1), columns=use_cols)
data_f_test = pd.DataFrame(np.concatenate((test_scaled, test_encoded), axis=1), columns=use_cols)

data_t = data_f.sample(frac=0.1)
data_f1 = data_f.drop(data_t.index, axis=0)

In [12]:
rf = RandomForestRegressor()
rf.fit(data_f1, target[data_f1.index])
rf_pred = rf.predict(data_t)

gb = GradientBoostingRegressor()
gb.fit(data_f1, target[data_f1.index])
gb_pred = gb.predict(data_t)

In [13]:
rf_pred1 = rf.predict(data_f1)
gb_pred1 = gb.predict(data_f1)

In [14]:
mae = mean_absolute_error(target[data_t.index], rf_pred)
r2 = r2_score(target[data_t.index], rf_pred)
mse = mean_squared_error(target[data_t.index], rf_pred)

mae_eval = mean_absolute_error(target[data_f1.index], rf_pred1)
r2_eval = r2_score(target[data_f1.index], rf_pred1)
mse_eval = mean_squared_error(target[data_f1.index], rf_pred1)


mae_gb = mean_absolute_error(target[data_t.index], gb_pred)
r2_gb = r2_score(target[data_t.index], gb_pred)
mse_gb = mean_squared_error(target[data_t.index], gb_pred)

mae_gb_eval = mean_absolute_error(target[data_f1.index], gb_pred1)
r2_gb_eval = r2_score(target[data_f1.index], gb_pred1)
mse_gb_eval = mean_squared_error(target[data_f1.index], gb_pred1)

data = {'Model': ['Random Forest', 'Gradient Boosting'],
        
        'Mean Absolute Error (MAE)': [mae_eval, mae_gb_eval],
        'R-Squared (R2)': [r2_eval, r2_gb_eval],
        'Mean Squared Error (MSE)': [mse_eval, mse_gb_eval],
        'Mean Absolute Error Eval (MAE)': [mae, mae_gb],
        'R-Squared (R2) Eval': [r2, r2_gb],
        'Mean Squared Error Eval(MSE)': [mse, mse_gb],
        }

# Buat dataframe dari dictionary
df = pd.DataFrame(data)
df

,Model,Mean Absolute Error (MAE),R-Squared (R2),Mean Squared Error (MSE),Mean Absolute Error Eval (MAE),R-Squared (R2) Eval,Mean Squared Error Eval(MSE)
0,Random Forest,6413.037838,0.979005,1.309477e+08,18486.235274,0.861366,9.591283e+08
1,Gradient Boosting,10549.034217,0.965978,2.121955e+08,16057.063090,0.879489,8.337444e+08


In [15]:
gb_f_pred = gb.predict(data_f_test)
ids = data_test.pop('Id')
submission = pd.DataFrame({'Id': ids, 'SalePrince': gb_f_pred.squeeze()})


In [16]:
submission.to_csv('final_submission.csv', index=False)